# Análise de Sentimento com uma RNN

Neste notebook, você irá implementar uma RNN que realiza análise de sentimentos.
>Usar uma RNN ao invés de uma rede estritamente feedforward é mais acurado visto que podemos incluir informações sobre a *sequência* de palavras.

Aqui, vamos usar um dataset de review de filmes, acompanhado por um rótulo de sentimento: positivo ou negativo

In [ ]:
from IPython.display import Image
Image(open('assets/reviews_ex.png','rb').read())

### Arquitetura da Rede

A Arquitetura da rede é mostrada abaixo

In [ ]:
from IPython.display import Image
Image(open('assets/network_diagram.png','rb').read())

>**Primeiro, vamos passar as palavras numa camada de embedding.** Precisamos de uma camada de embedding visto que temos milhares de palavras, então precisamos de uma representação eficiente que os vetores one-hot encode para nosso input de dados. Podemos treinar um modelo Word2Vec e usar esses embeddings como input. Entretanto, é interessante ter uma camada de embedding e deixar a rede aprender a tabela de embedding por conta própria. *Neste caso, a camada de embedding é para redução de dimensionalidade ao invés de ser para aprendizado de representação semântica.*

>**Depois, os inputs (palavras) são passados para uma camada de embeddings e os novos embeddings serão passados para células LSTM.** As células LSTM irão adicionar conexões *recorrentes* a rede e fornecer a habilidade de incluir informação sobre a *sequência* de palavras nos dados de review.

>**Finalmente, a saída da LSTM passará para uma sigmoide na camada de saída.** Estamos usando a função sigmoide porque tratamos positivo como 1 e negativo como 0 e a sigmoide produzirá valores entre 0-1.

Não nos preocuparemos com as saídas da sigmoide exceto pela **última**; podemos ignorar as demais. Iremos calcular a loss comparando o output no último instante de tempo e o rótulo da amostra.

---
### Carregar e visualizar os dados

In [ ]:
import numpy as np

# read data from text files
with open('data/reviews.txt', 'r') as f:
    reviews = f.read()
with open('data/labels.txt', 'r') as f:
    labels = f.read()

In [ ]:
print(reviews[:1000])
print()
print(labels[:18])

## Pré-processamento dos Dados

O primeiro passo ao construir um modelo de rede neural é colocar seus dados no formato adequado para input na rede. Visto que estamos usando camadas embedding, precisaremos encodar cada palavra como um inteiro. Vamos precisar também realizar alguma limpeza.

Estes são os passos do pré-processamento:

>* Queremos eliminar pontos e pontuações estranhas.
* Além disso, você pode ter notado que os reviews são delimitados com o caracter de nova linha `\n`. Para lidar com isso, vamos separar o texto em cada review usando `\n` como delimitador
* Então vamos combinar todos os reviews de volta numa única grande string.

Primeiro, vamos remover toda pontuação. Em seguida, pegue todo o texto sem novas linhas e divida-o em palavras individuais.

In [ ]:
from string import punctuation

# get rid of punctuation
reviews = reviews.lower() # lowercase, standardize
all_text = ''.join([c for c in reviews if c not in punctuation])

# split by new lines and spaces
reviews_split = all_text.split('\n')
all_text = ' '.join(reviews_split)

# create a list of words
words = all_text.split()

In [ ]:
words[:30]

### Codificando as palavras

A tabela de embedding requer que passemos números inteiros para nossa rede. A maneira mais fácil de fazer isso é criar dicionários que mapeiem as palavras do vocabulário para números inteiros. Em seguida, podemos converter cada um de nossos reviews em números inteiros para que possam ser transmitidas à rede.

In [ ]:
# feel free to use this import
from collections import Counter

## Build a dictionary that maps words to integers
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = []
for review in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])

**Testando**

Como um texto que você implementou o dicionário corretamente, imprima o número de palavras únicas em seu vocabulário e o conteúdo da primeira revisão tokenizada.

In [ ]:
# stats about vocabulary
print('Unique words: ', len((vocab_to_int)))  # should ~ 74000+
print()

# print tokens in first review
print('Tokenized review: \n', reviews_ints[:1])

### Codificando as labels

Nossos rótulos são “positivos” ou “negativos”. Para usar esses rótulos em nossa rede, precisamos convertê-los para 0 e 1.

In [ ]:
# 1=positive, 0=negative label conversion
labels_split = labels.split('\n')
encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels_split])

### Removendo Outliers

Como uma etapa adicional de pré-processamento, queremos ter certeza de que nossos reviews estão em um bom formato para o processamento padrão. Ou seja, nossa rede espera um tamanho de texto de entrada padrão e, portanto, queremos moldar nossos reviews em um comprimento específico. Abordaremos esta tarefa em duas etapas principais:

1. Livrar-se de reviews extremamente longos ou curtos; os outliers
2. Preencher/truncar (padding) os dados restantes para que tenhamos reviews do mesmo tamanho.

Before we pad our review text, we should check for reviews of extremely short or long lengths; outliers that may mess with our training.
Antes de preenchermos nosso review, devemos verificar se há reviews de comprimento extremamente curto ou longo; valores discrepantes que podem atrapalhar nosso treinamento.

In [ ]:
# outlier review stats
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Ok, alguns problemas aqui. Parece que temos um review com comprimento zero. E há um review muito grande para nossa RNN. Teremos que remover quaisquer reviews supercurtos e truncar reviews superlongos. Isso remove valores discrepantes e deve permitir que nosso modelo seja treinado com mais eficiência.

In [ ]:
print('Number of reviews before removing outliers: ', len(reviews_ints))

## remove any reviews/labels with zero length from the reviews_ints list.

# get indices of any reviews with length 0
non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review) != 0]

# remove 0-length reviews and their labels
reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

print('Number of reviews after removing outliers: ', len(reviews_ints))

---
## Padding sequences - ToDo 1

Para lidar tanto com reviews curtos quanto os muito grandes vamos preencher or truncar todos os reviews para um tamanho específico. Para reviews pequenos que uma determinada `seq_length`, vamos preencher com 0s. Para reviews maiores que `seq_length`, vamos truncar até o tamanho `seq_length`, que, neste caso é 200.

* Os dados devem vir de `review_ints`, visto que vamos alimentar a rede com números inteiros.
* Cada linha deve possuir comprimento igual a `seq_length`
* para reviews menores que `seq_length` palavras, preencha à esquerda com 0s (padding). Ex: se o review é `['best', 'movie', 'ever']`, ou `[117, 18, 128]` como inteiros, aplicando o padding o review será `[0, 0, 0, ..., 0, 117, 18, 128]`.
* Para reviews maiores que `seq_length`, use apenas as primeiras `seq_length` palavras.

Como um pequeno exemplo, se `seq_length=10` e review de input é
```
[117, 18, 128]
```
O vetor resultando, depois de aplicado o padding, deverá ser:
```
[0, 0, 0, 0, 0, 0, 0, 117, 18, 128]
```
**Seu array final `features` deve ser um array 2D, com o número de linhas iguais ao número de reviews e o número de colunas igual ao especificado em `seq_length`.**

Isto não é trivial e existem muitas maneiras de fazer isso, mas se você irá construir suas próprias redes Deep Learning, você terá que se acostumar a preparar os seus dados.

In [ ]:
def pad_features(reviews_ints, seq_length):
    '''
    Retorna features de reviews_ints, em que cada review é preenchido com 0s ou truncado até o tamanho de seq_length

    Dicas: crie uma matriz de tamanho mxn, em que m é a quantidade de reviews e n é seq_length e a preencha com zeros.
    Depois, percorra as linhas dessa matriz, capturando o tamanho de cada review (linha) e preencha, da ultima coluna para a primeira, com os reviews
    '''

    #START CODE HERE

    return features

In [ ]:
# Test your implementation!

seq_length = 200

features = pad_features(reviews_ints, seq_length=seq_length)

## test statements - do not change - ##
assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches
print(features[:30,:10])

## Training, Validation, Test
## Treino, Validação e Teste

Com os dados no formato correto, vamos dividi-lo em conjunto de treino, validação e teste.

In [ ]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape),
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

**Check your work**

With train, validation, and test fractions equal to 0.8, 0.1, 0.1, respectively, the final, feature data shapes should look like:
```
                    Feature Shapes:
Train set: 		 (20000, 200)
Validation set: 	(2500, 200)
Test set: 		  (2500, 200)
```

---
## DataLoaders e Batching

Depois de criar os conjuntos de treino, validação e teste, podemos criar os DataLoaders seguindos os seguintes passos:

1. Crie um formato para acessar nosso dados usando [TensorDataset](https://pytorch.org/docs/stable/data.html#) que toma como input um conjunto de dados e um conjunto de labels
2. Crie DataLoaders a partir dos Tensor datasets


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [ ]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = next(dataiter)

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

# Definição da Rede para análise de sentimento usando PyTorch - ToDo 2

Essa é a arquitetura da rede que vamos definir:

In [ ]:
from IPython.display import Image
Image(open('assets/network_diagram.png','rb').read())

---
As camadas são as seguintes:
1. Uma [camada embedding](https://pytorch.org/docs/stable/nn.html#embedding) que converte tokens de palavras (inteiros) em embeddings de tamanho específico.
2. Uma [camada LSTM](https://pytorch.org/docs/stable/nn.html#lstm) definida por um tamanho hidden_state e um número de camadas
3. uma camada de saída totalmente conectada que mapeia a saída da camada LSTM para um tamanho desejado
4. Uma camada com função de ativação sigmoide que transforma todos os outputs num valor 0-1; retorne **apenas a última saída da sigmoide** como saída da rede


### A camada de Embedding

Nós precisamos de uma camada de embedding porque temos mais de 74000 palavras em nosso vocabulário. É extremamente ineficiente fazer one-hot encoding desse valor e, por isso, criamos uma camada embedding que funcionará como uma tabela de pesquisa. Poderíamos treinar uma camada usando Word2Vec e usá-la aqui, mas é adequado criar uma nova camada, com o propósito de reduçãoo de dimensionalidade, e deixar a rede aprender os pesos.


### A camada LSTM

Vamos criar uma camada LSTM para usar em nossa rede recorrente, que toma um input_size, um hidden_dim e um número de camadas, uma probabilidade de dropout e um batch_first como parametros.


> **Exercício:** Complete as funções `__init__`, `forward`, e `init_hidden` na classe SentimentRNN.

In [ ]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

In [ ]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = 
        self.n_layers = 
        self.hidden_dim = 

        # embedding and LSTM layers
        self.embedding = 
        self.lstm = 

        # dropout layer
        self.dropout = 

        # linear and sigmoid layers
        self.fc = 
        self.sig = 


    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = 
        lstm_out, hidden = 

        lstm_out = lstm_out[:, -1, :] # getting the last time step output

        # dropout and fully-connected layer
        out = 
        out = 
        # sigmoid function
        sig_out = 

        # return last sigmoid output and hidden state
        return sig_out, hidden


    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())

        return hidden


## Instancie a Rede

Aqui, vamos instanciar a rede. Primeiro, vamos definir os hiperparâmetros.


* `vocab_size`: Tamanho do vocabulário
* `output_size`: Tamanho do output desejado
* `embedding_dim`: tamanho dos nossos embeddings
* `hidden_dim`: Número de neurônios na camada oculta. Geralmente 128, 256, 512, etc.
* `n_layers`: Número de camadas LSTM. Tipicamente entre 1-3


In [ ]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

---
## Treino

Abaixo está um típico código para treino. Vamos usar um tipo de loss que foi projetada para trabalhar com um único output da Sigmoide. [BCELoss](https://pytorch.org/docs/stable/nn.html#bceloss), ou **Binary Cross Entropy Loss**, aplica a loss cross entropy a um único valor entre 0 e 1.

Temos também alguns hiperparametros:

* `lr`: Learning rate
* `epochs`: Número de vezes a iterar sobre todo dataset.
* `clip`: O valor máximo de gradiente para corte (para prevenir explosão do gradiente) [link](https://machinelearningmastery.com/exploding-gradients-in-neural-networks/).

In [ ]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)


In [ ]:
# training params

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

---
## Teste

Exsitem algumas maneiras de testar nossa rede.

* **Performance no conjunto de teste:** Primeiro, veremos como nosso modelo treinado performa no conjunto de teste. Vamos calcular a loss e acurácia média.

* **Inferência em dados gerados pelo usuário:** Segundo, veremos como nosso modelo responde um review por vez (sem o rótulo) e analisaremos qual a predição do modelo treinado.

In [ ]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()

    # get predicted outputs
    output, h = net(inputs, h)

    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())

    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer

    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

### Inferência num review de teste

Você pode alterar o review para qualquer texto que queira. Leia-o e pense: é um review positivo ou negativo? Então veja se o seu modelo faz a predição correta.

In [ ]:
# negative test review
test_review_neg = 'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'


In [ ]:
from string import punctuation

def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int.get(word, 0) for word in test_words])

    return test_ints

# test code and generate tokenized review
test_ints = tokenize_review(test_review_neg)
print(test_ints)

In [ ]:
# test sequence padding
seq_length=200
features = pad_features(test_ints, seq_length)

print(features)

In [ ]:
# test conversion to tensor and pass into your model
feature_tensor = torch.from_numpy(features)
print(feature_tensor.size())

In [ ]:
def predict(net, test_review, sequence_length=200):

    net.eval()

    # tokenize review
    test_ints = tokenize_review(test_review)

    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)

    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)

    batch_size = feature_tensor.size(0)

    # initialize hidden state
    h = net.init_hidden(batch_size)

    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()

    # get the output from the model
    output, h = net(feature_tensor, h)

    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())
    # printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))

    # print custom response
    if(pred.item()==1):
        print("Positive review detected!")
    else:
        print("Negative review detected.")


In [ ]:
# positive test review
test_review_pos = 'This movie had the best acting and the dialogue was so good. I loved it.'


In [ ]:
# call function
seq_length=200 # good to use the length that was trained on

predict(net, test_review_neg, seq_length)